In [1]:
from __future__ import print_function, division
import time, os
import numpy as np
import matplotlib.pyplot as plt
import sys
#import networkx as nx
import pandas
import math

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [26]:
# load cleaned data
#selected_snapshots.read_csv('/home/cs231n/data/snapshots_cleaned.csv')
#selected_repairs.read_csv('/home/cs231n/data/repairs_cleaned.csv')
selected_snapshots = pandas.read_pickle('cleaned_selected_snapshots.pkl')
selected_repairs = pandas.read_pickle('cleaned_selected_repairs.pkl')

# Reset indices of loaded dataframes
selected_snapshots = selected_snapshots.reset_index(drop=True)
selected_repairs = selected_repairs.reset_index(drop=True)

In [3]:
def get_012_dates(end_date):
    ## split into 0 (10+), 1 (5-10), 2(0-5)
    two_end = end_date
    one_end = pandas.to_datetime(two_end) + pandas.DateOffset(days=-5)
    zero_end = one_end + pandas.DateOffset(days=-5)
    return (zero_end, one_end, two_end)

In [30]:
#-------------------------------------------------------------
# Label Snapshots: creates a new column 'state' in selected_snapshots and populates the state
# 0: 10 or more days before repair date
# 1: 5 - 9 days
# 2: < 5 days before repair date
#-------------------------------------------------------------
# Create a new column 'state' for the label of each snapshot
selected_snapshots['Model Vehicle'] = 'N.A.'
selected_snapshots['ATA3'] = 'N.A.'
selected_snapshots['ATA6'] = 'N.A.'
selected_snapshots['ATA9'] = 'N.A.'
selected_snapshots['Repair Cost'] = 'N.A.'
selected_snapshots['Repair Date'] = 'N.A.'
#selected_snapshots['daysToNextRepair'] = np.nan
selected_snapshots['state'] = 0
selected_repairs['snapshot Qty b4 RepairDt'] = np.nan

# List to count #snapshots before each repair date
snapshot_size = []
repairs = selected_repairs[selected_repairs['Repair Cost'].isin(['very high', 'high ', 'medium'])]

# Label state of each snapshot as 0, 1 or 2
for veh_id in repairs['Chassis\nReference\nNumber'].unique():
    v_snapshots = selected_snapshots[selected_snapshots['Veh Ref ID'] == veh_id].sort_values(by='Event DateTime')
    v_repairs = repairs[repairs['Chassis\nReference\nNumber'] == veh_id].sort_values(by='Rpr_Dt')
    
    # transfer 'Model Vehicle' from repair data to snapshots data for each vehicle
    v_model = ''.join(e for e in str(v_repairs['Model Vehicle'].unique()) if e.isalnum())
    selected_snapshots.loc[v_snapshots.index,'Model Vehicle'] = v_model
    
    start = pandas.to_datetime('1/1/2000') ## in past so first snapshot is captured
    event_dt_key = 'Event DateTime'
    state_key = 'state'
    
    for i in v_repairs.index: # v_repairs has been sorted by 'Rpr_Dt'
        repairDate = v_repairs.loc[i]['Rpr_Dt']
        mask = (v_snapshots[event_dt_key] > start) & (v_snapshots[event_dt_key] <= repairDate)
        snapshot_size.append({'Chassis\nReference\nNumber':veh_id, 'Rpr_Dt':repairDate, 'Snapshot Qty before Rpr_Dt':sum(mask)})
        selected_repairs.loc[i,'snapshot Qty b4 RepairDt'] = sum(mask)
        #if (sum(mask) == 0):
        #    continue
        
        # transfer 'ATA3', 'ATA6', 'ATA9' and 'Repair Cost' from repair data to snapshots data        
        v_ATA3 = ''.join(e for e in str(v_repairs.loc[i]['ATA3']) if e.isalnum())
        v_ATA6 = ''.join(e for e in str(v_repairs.loc[i]['ATA6']) if e.isalnum())
        v_ATA9 = ''.join(e for e in str(v_repairs.loc[i]['ATA9']) if e.isalnum())
        v_repairCost = str(v_repairs.loc[i]['Repair Cost'])
        index = v_snapshots.index[(v_snapshots[event_dt_key] > start) & (v_snapshots[event_dt_key] <= repairDate)]
        selected_snapshots.loc[index,'ATA3'] = v_ATA3
        selected_snapshots.loc[index,'ATA6'] = v_ATA6
        selected_snapshots.loc[index,'ATA9'] = v_ATA9
        selected_snapshots.loc[index,'Repair Cost'] = v_repairCost
        selected_snapshots.loc[index,'Repair Date'] = repairDate

        #for j in index:            
        #    selected_snapshots.loc[j,'daysToNextRepair'] = (repairDate - selected_snapshots.loc[j,'Event DateTime']).days
        
        # divide up snapshots into 0, 1, 2 slices and initialize corresponding state for each snapshot as 0, 1, 2 respectively
        (zero_end, one_end, two_end) = get_012_dates(repairDate)
        two_index = v_snapshots.index[(v_snapshots[event_dt_key] > one_end) & (v_snapshots[event_dt_key] <= two_end)]
        one_index = v_snapshots.index[(v_snapshots[event_dt_key] > zero_end) & (v_snapshots[event_dt_key] <= one_end)]
        zero_index = v_snapshots.index[(v_snapshots[event_dt_key] >= start) & (v_snapshots[event_dt_key] <= zero_end)]
        selected_snapshots.loc[two_index,state_key] = 2
        selected_snapshots.loc[one_index,state_key] = 1
        selected_snapshots.loc[zero_index,state_key] = 0
        
        ## reset start to repairDate to initialize states until next repairDate
        start = repairDate
    
    # initialize states beyond last repair date to 'np.nan'
    other_index = v_snapshots.index[v_snapshots[event_dt_key] > repairDate]
    selected_snapshots.loc[other_index,state_key] = np.nan

# Tabulate #snapshots before repair date for the selected repairs (i.e. medium/high/very high cost)
pandas.DataFrame(snapshot_size)

,Chassis Reference Number,Rpr_Dt,Snapshot Qty before Rpr_Dt
0,208.0,2016-02-18,80
1,761.0,2016-03-08,109
2,765.0,2016-03-07,209
3,60.0,2016-03-31,969
4,182.0,2016-04-26,288
5,543.0,2016-03-29,218
6,319.0,2016-04-13,4
7,27.0,2016-04-08,8
8,460.0,2016-03-04,35
9,18.0,2016-04-25,238


In [32]:
#------------------------------------------------------------
# Data Cleaning - Part 2 
#------------------------------------------------------------
# Datatype Assignement - selected_snapshot
# Assign correct datatype for categorical variables in selected_snapshots     
category_var = ['Veh Ref ID','Event Type Description','Brake Switch','Clutch Switch','Cruise Status','Dpf Regen Inhibit Sw', 
                'Dpf Thermal Mngmnt','Eng Coolant Level','Population','DTCID','state',
                'Model Vehicle','ATA3','ATA6','ATA9','Repair Cost']

for i in category_var:
    selected_snapshots[i] = selected_snapshots[i].astype('category')
    
header = list(selected_snapshots.columns.values)
print('Feature:\tDatatype\t#Unique Entries')
for i in header:
    print('%s:\t%s\t%s' % (i, selected_snapshots[i].dtype, len(selected_snapshots[i].unique())))

# Drop dominant column 'Polulation': only 1 unique value ('2') for all snapshots
selected_snapshots.drop('Population', axis=1, inplace=True)

Feature:	Datatype	#Unique Entries
Veh Ref ID:	category	797
Event DateTime:	datetime64[ns]	956362
Event Type Description:	category	4
Acc Pedal Position:	float64	251
Ambient Air Temp:	float64	696
Barometric Press:	float64	76
Brake Switch:	category	2
Bus Utilization:	float64	83
Cat Intake Gas Temp:	float64	4462
Cat Outlet Gas Temp:	float64	5152
Clutch Switch:	category	2
Cmd Eng Fuel Press:	float64	20405
Cruise Status:	category	2
Dpf Regen Inhibit Sw:	category	3
Dpf Thermal Mngmnt:	category	2
Drvr Demand Torque:	float64	102
Eng Air Flow Rate:	float64	18800
Eng Avg Fuel Econ:	float64	4129
Eng Coolant Level:	category	3
Eng Coolant Temp:	float64	129
Eng Demand Torque:	float64	104
Eng DPF Intake Press:	float64	63
Eng Egr Valve Pos:	float64	1001
Eng Exhaust Gas Temp:	float64	601
Eng Fuel Del Press:	float64	255
EngFuelTemp1:	float64	102
Engine Speed:	float64	6353
Eng Man Abs Pressure:	float64	159
Eng Oil Pressure:	float64	196
EngInjRail1Press:	float64	2472
EngIntakeMan1Temp:	float64	126
EngOilTe

In [33]:
def split_test_train(repairs):
    mask = (repairs['Chassis\nReference\nNumber']==0) #all false
    
    for repairCode in repairs['ATA9'].unique():    
        v_repairs = repairs[repairs['ATA9'] == repairCode]
        idx = v_repairs.index
        print(repairCode, len(v_repairs))

        if len(v_repairs) > 1:
            # randomly select N cases with matching ATA9 code
            # where N = maxTestCase ~ 20% of #cases with the same ATA9 code
            maxTestCase = math.ceil(0.2*len(v_repairs))
            a = list(range(0,maxTestCase))
            np.random.shuffle(a)
            mask.loc[idx[a[0:maxTestCase]]] = True

    #mask.loc[idx[a[0:maxTestCase]]]
    test = repairs[mask][['Chassis\nReference\nNumber','Model Vehicle','ATA3','ATA6','ATA9','Repair Cost','Rpr_Dt','snapshot Qty b4 RepairDt']].sort_values('ATA9')
    train = repairs[~mask][['Chassis\nReference\nNumber','Model Vehicle','ATA3','ATA6','ATA9','Repair Cost','Rpr_Dt','snapshot Qty b4 RepairDt']].sort_values('ATA9')
    
    #print([test, train])
    return [test, train]

In [37]:
very_high_repairs = selected_repairs[selected_repairs['Repair Cost'].isin(['very high'])]
[veryhigh_test, veryhigh_train] = split_test_train(very_high_repairs)
#[veryhigh_test, veryhigh_train]

45002000 4
45007001 2
45002072 3
43006007 1


In [38]:
high_repairs = selected_repairs[selected_repairs['Repair Cost'].isin(['high '])]
[high_test, high_train] = split_test_train(high_repairs)
#[high_test, high_train]

45007001 3
45002072 2
44003016 1
43004000 2
45008098 2
44004001 1


In [39]:
medium_repairs = selected_repairs[selected_repairs['Repair Cost'].isin(['medium'])]
[medium_test, medium_train] = split_test_train(medium_repairs)
#[medium_test, medium_train]

44003217 1
43001071 1
45021003 12
44004001 41
45007031 1
45018000 2
43001058 2
45007001 1
44003027 2
45013001 1
43001049 2
45002072 1
13009001 1
101001001 1
43007023 1
44014014 2
3001082 1
43006007 1
45007007 1
45002000 1
43007096 1


In [40]:
test = veryhigh_test
train = veryhigh_train
test = test.append(high_test)
test = test.append(medium_test)
train = train.append(high_train)
train = train.append(medium_train)
[len(test), len(train)]

[24, 74]

In [41]:
#-------------------------------------------------------------
# Define: ok = no repairs with medium, high or very high cost
# Split "ok" vehicles into ok_test, ok_train for inclusion in test and train (if necessary)
#-------------------------------------------------------------
# Get "ok" vehicle IDs
ok_veh_ids = list(set(selected_snapshots['Veh Ref ID'].unique()) - set(very_high_repairs['Chassis\nReference\nNumber'].unique()) 
                 - set(high_repairs['Chassis\nReference\nNumber'].unique())
                  - set(medium_repairs['Chassis\nReference\nNumber'].unique()) )
ok_veh_ids = (np.array(ok_veh_ids))

# Get random permutation of indices of ok_veh_ids
a = pandas.Series(np.random.random(len(ok_veh_ids)))
a = a.sort_values().index

# split ok vehicles into test and train with the same proportion as those with medium, high & very high repair Cost 
test_train_proportion = len(test)/(len(test)+len(train))
maxTestCase = round(len(ok_veh_ids)*test_train_proportion)
ok_test = ok_veh_ids[a[0:maxTestCase].values]
ok_train = ok_veh_ids[a[maxTestCase:]]

print([len(ok_test),len(ok_train)])

[173, 532]


In [42]:
#drop the rows with eventDates beyond last repair date (i.e. state = 'np.nan')
selected_snapshots = selected_snapshots.dropna()

# sort snapshots first by vehicle ID, then by Event DateTime
selected_snapshots = selected_snapshots.sort_values(by=['Veh Ref ID','Event DateTime'])

# Reset indices after dropping NA rows
selected_snapshots = selected_snapshots.reset_index(drop=True)

In [43]:
# Split snapshots into test_snapshots and train_snapshots (snapshots of 'ok' vehicles excluded for now)
#------------------------------------------------------------
test_snapshots = selected_snapshots[selected_snapshots['Veh Ref ID'].isin(test['Chassis\nReference\nNumber'].unique())]
train_snapshots = selected_snapshots[selected_snapshots['Veh Ref ID'].isin(train['Chassis\nReference\nNumber'].unique())]
ok_test_snapshots = selected_snapshots[selected_snapshots['Veh Ref ID'].isin(ok_test)]
ok_train_snapshots = selected_snapshots[selected_snapshots['Veh Ref ID'].isin(ok_train)]
[test_snapshots.shape, train_snapshots.shape, ok_test_snapshots.shape, ok_train_snapshots.shape]

[(9325, 70), (41993, 70), (233063, 70), (642603, 70)]

In [54]:
# save cleaned data to pickle format for quick reloading in future
test_snapshots.to_pickle('test_snapshots.pkl')
train_snapshots.to_pickle('train_snapshots.pkl')
test.to_pickle('test.pkl')
train.to_pickle('train.pkl')

ok_test_snapshots.to_pickle('ok_test_snapshots.pkl')
ok_train_snapshots.to_pickle('ok_train_snapshots.pkl')
pandas.DataFrame(ok_test).to_pickle('ok_test.pkl')
pandas.DataFrame(ok_train).to_pickle('ok_train.pkl')